# Multi-class Classification with Softmax

## 1. Softmax Cost function with Pytorch

소프트맥스 함수는 다음과 같이 정의 된다.
$$\sigma(\mathbf{z})_i=\frac{exp({z_i})}{\sum_{j=1}^{K}exp({z_j})}$$

위의 식은 $K$의 개의 classes가 있을 때 $i$ 주어진 $\mathbf{z}$ (보통 각 클래스에 해당하는 scores를 element로 갖는 벡터)를 확률로 바꾸기 위해서 softmax 를 사용한다. (scores -> [0, 1] 범위의 값들로 변환)

In [1]:
import torch
import torch.nn.functional as F

In [2]:
z = torch.FloatTensor([1, 2, 3])

In [3]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
hypothesis.sum()

tensor(1.)

이제 cost function을 구현

Cross Entropy

일반적으로 Multi-class classification을 할 때는 Cross Entropy를 사용한다.
식은 다음과 같다.
$$\mathcal{L}=-\sum_{j=1}^{K}y_j\log (p_j)$$
여기에서 $K$는 클래스 갯수 이고 $y_j$는 $\mathbf{y}$라는 one-hot vector label의 $j$번째 element를 뜻 하고 $p_j$는 softmax output의 $j$ 번째 element 값을 뜻 한다.

이것을 N개의 전체 데이터에 대해서 확장 시키면 다음과 같다.
$$\mathcal{L}=-\frac{1}{N}\sum_{i=1}^{N} \sum_{j=1}^{K}y_j^{(i)}\log (p_j^{(i)})$$

Binary Classification과의 관계
Logistic Regression에서의 cost function을 다시 써 보면 다음과 같다.
$$\mathcal{L}=-\left(y\log H(x) + (1-y)\log (1-H(x)\right)$$
이것을 cross entropy 관점에서 해석 할 수 있다.

- $y=y_1$
- $1-y=y_2$
- $H(x)=p_1$
- $1-H(x)=p_2$

라고 해석하면 $K=2$인 경우로 볼 수 있다. (Cross Entropy의 Special Case)

$$\mathcal{L}=-\frac{1}{N}\sum_{i=1}^{N}\sum_{j=1}^{2}y_j^{(i)}\log (p_j^{(i)})= -\frac{1}{N}\sum_{i=1}^{N}\left(y^{(i)}\log (p^{(i)})+(1-y^{(i)})\log (1-p^{(i)})\right)$$

In [5]:
z = torch.rand(3, 5, requires_grad=True)

In [10]:
F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1591, 0.2278, 0.2215, 0.1459, 0.2457],
        [0.1610, 0.2088, 0.2699, 0.1309, 0.2294],
        [0.2486, 0.1642, 0.1506, 0.2251, 0.2115]], grad_fn=<SoftmaxBackward>)


In [11]:
# Label 생성
y = torch.randint(high=5, size=(3,)).long()
print(y)

tensor([1, 0, 2])


In [12]:
# One-hot encoding
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(dim=1, index=y.unsqueeze(1), value=1)

tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [13]:
y.unsqueeze(1)

tensor([[1],
        [0],
        [2]])

In [14]:
print(y_one_hot)

tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])


In [15]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.7330, grad_fn=<MeanBackward0>)


다른 함수 활용하기

1. F.softmax() + torch.log() = F.log_softmax()

In [16]:
# 처음 구현
torch.log(F.softmax(z, dim=1))

tensor([[-1.8383, -1.4794, -1.5073, -1.9245, -1.4037],
        [-1.8265, -1.5665, -1.3098, -2.0331, -1.4721],
        [-1.3917, -1.8069, -1.8930, -1.4913, -1.5536]], grad_fn=<LogBackward>)

In [17]:
# F.log_softmax() 활용
torch.log_softmax(z, dim=1)

tensor([[-1.8383, -1.4794, -1.5073, -1.9245, -1.4037],
        [-1.8265, -1.5665, -1.3098, -2.0331, -1.4721],
        [-1.3917, -1.8069, -1.8930, -1.4913, -1.5536]],
       grad_fn=<LogSoftmaxBackward>)

2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [18]:
# 첫 번째 구현
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.7330, grad_fn=<MeanBackward0>)

In [19]:
# 두 번쨰 구현
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.7330, grad_fn=<MeanBackward0>)

In [20]:
# 세번째 구현
# F.nll_loss()를 사용하면 one-hot vector를 넣을 필요 없음.
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.7330, grad_fn=<NllLossBackward>)

In [21]:
# 네번째 구현
F.cross_entropy(z, y)

tensor(1.7330, grad_fn=<NllLossBackward>)

## 2. Multi-class Classification Implementation

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [23]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
# classes: 0, 1, 2
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [24]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


### 2.1. Low-level Implementation

In [25]:
# One-hot Encoding
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [26]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [27]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### 2.2 High-level Implementation

F.cross_entropy() 사용하기

In [28]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{nb_epochs} Cost: {cost.item(): .6f}")

Epoch 0/1000 Cost:  1.098612
Epoch 100/1000 Cost:  0.761050
Epoch 200/1000 Cost:  0.689991
Epoch 300/1000 Cost:  0.643229
Epoch 400/1000 Cost:  0.604117
Epoch 500/1000 Cost:  0.568255
Epoch 600/1000 Cost:  0.533922
Epoch 700/1000 Cost:  0.500291
Epoch 800/1000 Cost:  0.466908
Epoch 900/1000 Cost:  0.433507
Epoch 1000/1000 Cost:  0.399962


### 2.3. Implementation with nn.Module

In [30]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

In [31]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{nb_epochs} Cost: {cost.item(): .6f}")

Epoch 0/1000 Cost:  3.281332
Epoch 100/1000 Cost:  0.690425
Epoch 200/1000 Cost:  0.607049
Epoch 300/1000 Cost:  0.549280
Epoch 400/1000 Cost:  0.498466
Epoch 500/1000 Cost:  0.450477
Epoch 600/1000 Cost:  0.403753
Epoch 700/1000 Cost:  0.357587
Epoch 800/1000 Cost:  0.311905
Epoch 900/1000 Cost:  0.268670
Epoch 1000/1000 Cost:  0.241308


### 2.4 Implementatio with class

[연습]

- Data: x_train, y_train

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        # Your Code
        super().__init__()
    
    def forward(self, x):
        # Your Code

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # Your Code
    prediction = model(x_train)

    # cost 계산
    # Your Code
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

## 3. MNIST Classification with Linear Model

In [32]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
print(device)

cuda


In [35]:
# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [36]:
# hyperparameters
training_epochs = 15
batch_size = 100

MNIST Classifier

In [39]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

/home/hyunjun/anaconda3/envs/rs/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630839582/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [40]:
# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                         batch_size=batch_size, # 배치 크기는 100
                         shuffle=True,
                         drop_last=True)

In [41]:
# MNIST data image of shape 28 * 28 = 784
linear = nn.Linear(784, 10, bias=True).to(device)

In [42]:
# Obejective function and Optimizer
criterion = nn.CrossEntropyLoss() # 내부적으로 소프트맥스 함수를 포함하고 있음.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [43]:
for epoch in range(training_epochs): # 앞서 training_epochs의 값은 15로 지정함.
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.view(-1, 28 * 28).to(device)
        # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.
        Y = Y.to(device)

        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.535146952
Epoch: 0002 cost = 0.359576732
Epoch: 0003 cost = 0.331263363
Epoch: 0004 cost = 0.316403836
Epoch: 0005 cost = 0.307106405
Epoch: 0006 cost = 0.300456613
Epoch: 0007 cost = 0.294932663
Epoch: 0008 cost = 0.290955693
Epoch: 0009 cost = 0.287074119
Epoch: 0010 cost = 0.284515440
Epoch: 0011 cost = 0.281915218
Epoch: 0012 cost = 0.279527277
Epoch: 0013 cost = 0.277636498
Epoch: 0014 cost = 0.275875330
Epoch: 0015 cost = 0.274422616
Learning finished


Class를 이용해서 코딩 + Custom DataLoader 사용해 보기

In [ ]:
import torch
from torchvision import transforms

# Your Code
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        # Your Code
    
    def __len__(self):
        # Your Code
    
    def __getitem__(self, idx):
        # Your Code

In [ ]:
x_train = mnist_train.data / 256.
y_train = mnist_train.targets

In [ ]:
# Your Code
mnist_train_data = 
data_loader = 

In [ ]:
# Your Code
class MultiClassLinear(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        # Your Code

In [ ]:
model = MultiClassLinear().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.01)

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
    
    for x, y in data_loader:
        x = x.view(-1, 28 * 28).to(device)
        y = y.to(device)
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print(f"Epoch {epoch+1} Cost: {avg_cost:.9f}")
print('Learning finished')